In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd "/content/gdrive/MyDrive/Colab Notebooks/"

/content/gdrive/MyDrive/Colab Notebooks


In [ ]:
!ls

'=1.6.0'
 1712396_AssignmentML
 1712396_BT1_KiemTraHamSinhSo.ipynb
 1712396_BT2_MNIST-Classification-pytorch.ipynb
 1712396_BTL_GANforCIFAR_10.ipynb
 1712396_BTL_VAEforCIFAR_10.ipynb
 annotations
 bottom-up-feat.ipynb
 data1M300d.npz
 demo_faster_rcnn.ipynb
 demo.ipynb
 DoggyTask.ipynb
 en-vi.ipynb
 envi-model.avg-250000.tar.xz
 en.wiki.bpe.vs10000.model
 glove.6B.100d.txt
 glove.6B.200d.txt
 glove.6B.300d.txt
 glove.6B.50d.txt
 glove.6B.zip
 gnmt_en_vi_u512
 ImagePresentation.ipynb
 MNIST-cnn-pytorch.ipynb
 model_en_vi_epoch19_9.12.t7
 netflix_titles.csv.gsheet
 nmt
 NMT_icode.ipynb
 output2021-03-16.csv.gsheet
 sentencepiece
 t2t_export
 Task1WhattowatchonNetflix.ipynb
 test-2013-en-vi.tgz
 train_answer.txt
 train_en.txt
 training_checkpoints
 train_question.txt
 train_vi.txt
 translate.ipynb
 TranslationEn-Vi.ipynb
 Translation.ipynb
 tst2013.en
 tst2013.vi
 Tut1-Pytorch.ipynb
 Tut2.ipynb
 Tut3.ipynb
 Tut4.ipynb
 Untitled
 Untitled0.ipynb
 Untitled1.ipynb
 Untitled2.ipynb
 Untitled3

In [ ]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove*.zip

In [ ]:
import torch
import pandas as pd
import numpy as np
from string import punctuation
from collections import Counter
import matplotlib.pyplot as plt
import string 
import os
import re
%matplotlib inline

In [ ]:
embedding_dict = dict()
f = open('glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coef = np.asarray(values[1:],dtype='float32')
    embedding_dict[word] = coef # save word as key and coefficients as the value in the dictionary
f.close()

In [ ]:
def preprocess(file):
    processed = [] # preprocess the file
    lines = file.read().split('\n')
    for line in lines:
        line = line.lower() # lower case
        line = line.replace('#','<num>') # replace the token '#' with '<num>'
        line = re.sub(">.<",' point ',line) # replace the token '>.<' with ' point '
        text = re.sub(r"[^a-z?.!,'<>]+"," ",line) # replace other tokens with a space
        text = text.rstrip().strip() # strip white space
        text = 'bos ' + text + ' eos' # beginning and end tokens for each sentence
        processed.append(text)
    return processed

In [ ]:

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('output2021-03-16.csv').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
rows = rows[1:]
# Convert to a DataFrame and render.
import pandas as pd
pd.DataFrame.from_records(rows)

,0,1
0,When was the Vat formally opened?,Formally established in 1475
1,what is the library for?,he Vatican Library is a research library
2,for what subjects?,Vatican Library is a research library for hist...
3,and?,Vatican Library is a research library for hist...
4,what was started in 2014?,"March 2014, the Vatican Library began an initi..."
...,...,...
108642,Who was a sub?,substitute Xabi Alonso
108643,Was it his first game this year?,Xabi Alonso made his first appearance of the s...
108644,What position did the team reach?,Real moved up to third in the table
108645,Who was ahead of them?,six points behind Barca.


In [ ]:
lines_train_question = np.array(rows)[:,0]
lines_train_answer = np.array(rows)[:,1]

In [ ]:
TRAIN_ques_len = []
TRAIN_ans_len = [] 

for line in lines_train_question:
    w = 0
    for word in line.split(' '):
        w += 1
    TRAIN_ques_len.append(w)

for line in lines_train_answer:
    w = 0
    for word in line.split(' '):
        w += 1
    TRAIN_ans_len.append(w)
TRAIN_ques_len = np.array(TRAIN_ques_len)
TRAIN_ans_len = np.array(TRAIN_ans_len) 

In [ ]:
print("\t\t Min\t 25%\t Avg\t 75%\t Max")
print("\nTRAIN Input\t {}\t{}\t{}\t{}\t{}".format(np.percentile(TRAIN_ques_len,0),
                                                  np.percentile(TRAIN_ques_len,25),
                                                  np.percentile(TRAIN_ques_len,50),
                                                  np.percentile(TRAIN_ques_len,75),
                                                  np.percentile(TRAIN_ques_len,100)))
print("TRAIN Summary\t {}\t{}\t{}\t{}\t{}".format(np.percentile(TRAIN_ans_len,0),
                                                  np.percentile(TRAIN_ans_len,25),
                                                  np.percentile(TRAIN_ans_len,50),
                                                  np.percentile(TRAIN_ans_len,75),
                                                  np.percentile(TRAIN_ans_len,100)))

		 Min	 25%	 Avg	 75%	 Max

TRAIN Input	 1.0	4.0	5.0	7.0	45.0
TRAIN Summary	 1.0	3.0	7.0	12.0	353.0


In [ ]:
words = [] 
for line in lines_train_question:
    for word in line.split(' '):
        words.append(word)
for line in lines_train_answer:
    for word in line.split(' '):
        words.append(word)
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
embed_dim = 300
words_found = 0
words_OOV = 0
word2idx = {}
word2idx['<pad>'] = 0
# convert words to index by creating word2idx dictionary
for index, word in enumerate(vocab):
    word2idx[word] = index+1
# create weight embedding matrix for GloVe embedding
weight_embedding = np.zeros([len(vocab)+1,embed_dim],dtype='float32')
for i, word in enumerate(vocab):
    try:
        # if word found in pretrained GloVe embedding dictionary
        weight_embedding[i,:] = embedding_dict[word]
        words_found += 1
    except:
        # if word not found, initialize as random vector
        weight_embedding[i,:] = np.random.normal(scale=0.6, size=[embed_dim,])
        words_OOV += 1

print("{} words found".format(words_found))
print("{} OOV words".format(words_OOV))
idx2word = dict((v,k) for k,v in word2idx.items())
print("{} words in the vocabulary".format(len(vocab)))

26435 words found
76799 OOV words
103234 words in the vocabulary


In [ ]:
ques_ints = []
ans_ints = []

for each in lines_train_question:
    sentence = []
    for word in each.split():
        sentence.extend([word2idx[word]])
    ques_ints.append(sentence)

for each in lines_train_answer:
    sentence = []
    for word in each.split():
        if word in word2idx:
          sentence.extend([word2idx[word]])
        else:
          word = word.lower()
          if word in word2idx:
            sentence.extend([word2idx[word]])
          else:
            if word == 'eos':
              sentence.extend([word2idx['end']])
            else:
              sentence.extend([word2idx['unknown']])
    ans_ints.append(sentence)

ques_len = 50 
ans_len = 14
ques_feature = np.zeros((len(ques_ints),ques_len),dtype=int)
ans_feature = np.zeros((len(ans_ints),ans_len), dtype=int)
for i, row in enumerate(ques_ints):
    ques_feature[i,:len(row)] = np.array(row)[:ques_len]
for i, row in enumerate(ans_ints):
    ans_feature[i,:len(row)] = np.array(row)[:ans_len]

# train test split
split_frac = 0.85 
split_index = int(len(ques_feature)*0.85)
train_ques, test_ques = ques_feature[:split_index],ques_feature[split_index:]
train_ans, test_ans = ans_feature[:split_index],ans_feature[split_index:]

print("\t\t\tData Shapes:")
print("Train set: \t\t{}".format(train_ques.shape), 
      "\nTest set: \t\t{}".format(test_ques.shape))

print("\n\n\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_ans.shape),
      "\nTest set: \t\t{}".format(test_ans.shape))

			Data Shapes:
Train set: 		(92349, 50) 
Test set: 		(16298, 50)


			Feature Shapes:
Train set: 		(92349, 14) 
Test set: 		(16298, 14)


In [ ]:
np.savez('data1M300d.npz',train_ques=train_ques,test_ques=test_ques,
         train_ans=train_ans,test_ans=test_ans,
         word2idx = word2idx, idx2word=idx2word, 
         embed_matrix=weight_embedding, output_length = TRAIN_ans_len)
print("Data Saved!")

Data Saved!


In [ ]:
import torch
import pandas as pd
import numpy as np
from string import punctuation
from collections import Counter
import matplotlib.pyplot as plt
import string 
import os
import re
import queue
%matplotlib inline

# train_ques = data
# test_ques = data['test_ques']
# train_summary = data['train_summary']
# test_summary = data['test_summary']
# word2idx = data['word2idx']
# word2idx = dict(word2idx.item())
# idx2word = data['idx2word']
# idx2word = dict(idx2word.item())
# weight_embedding = data['embed_matrix']
# weight_embedding = torch.from_numpy(weight_embedding)
# output_length = data['output_length']
split_index = int(len(test_ques)*0.1)
val_ques, test_ques  = test_ques[:split_index],test_ques[split_index:]
val_ans, test_ans = test_ans[:split_index],test_ans[split_index:]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device to run the code on GPU


In [ ]:
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F


def create_emb(weight_matrix, non_trainable=False):
    # create embedding matrix with the pretrained GloVe embedding matrix
    emb_layer = torch.nn.Embedding(weight_matrix.shape[0],weight_matrix.shape[1])
    emb_layer.load_state_dict({'weight': weight_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False # turn off training for the embedding vector
    return emb_layer


class Encoder(nn.Module):
    def __init__(self,kernel_size,filter_size,dropout,num_hidden,layers,weight_matrix,embed_dim,device):
        super(Encoder,self).__init__()
        self.layers = layers
        self.kernel_size = kernel_size
        self.filter_size = filter_size
        self.num_hidden = num_hidden
        self.device = device
        self.embed_dim = embed_dim
        
        # convolutional layers
        self.conv1 = torch.nn.Conv1d(embed_dim,self.filter_size,self.kernel_size[0],stride=1,padding=0)
        self.conv2 = torch.nn.Conv1d(embed_dim,self.filter_size,self.kernel_size[1],stride=1,padding=1)
        self.conv3 = torch.nn.Conv1d(embed_dim,self.filter_size,self.kernel_size[2],stride=1,padding=2)
        self.dropout = torch.nn.Dropout(dropout)
        self.embedding = create_emb(weight_matrix,True)
        # encoder LSTM
        self.lstm = torch.nn.LSTM(input_size = self.embed_dim,hidden_size = self.num_hidden,
                                  num_layers = self.layers,batch_first=True,dropout=0.5,
                                 bidirectional=True)
    def forward(self,x,hidden):
        x = self.embedding(x) # embed the input text
        
        # convolutional layers
        #x1 = torch.tanh(self.dropout(self.conv1(x)))
        #x2 = torch.tanh(self.dropout(self.conv2(x)))
        #x3 = torch.tanh(self.dropout(self.conv3(x)))
        # apply dropout
        lstm_in = self.dropout((x))
        # encoder LSTM
        output,(h_hidden,c_hidden) = self.lstm(lstm_in,hidden)
        # hidden state of the encoder (in case of multilayer LSTM)
        h = torch.cat((h_hidden[-1,:,:],h_hidden[-2,:,:]),1)
        c = torch.cat((c_hidden[-1,:,:],c_hidden[-2,:,:]),1)

        hidden = (h.unsqueeze(0),c.unsqueeze(0))
       
        return output, hidden
    
    def init_hidden(self,batch_size):
        # initialize the hidden state as the zero tensor
        weight = next(self.parameters()).data
        hidden = (weight.new(self.layers*2,batch_size, self.num_hidden).zero_(),
                  weight.new(self.layers*2, batch_size, self.num_hidden).zero_())
        return hidden
        

In [ ]:
class AttentionDecoder(nn.Module):
    def __init__(self,num_hidden,dropout,vocab_size,layers,weight_matrix,embed_dims,device):
        super(AttentionDecoder,self).__init__()
        self.num_hidden = num_hidden
        self.dropout = dropout
        self.layers = layers
        self.vocab_size = vocab_size
        self.embed_dims = embed_dims
        self.device = device
        self.dropout_layer = torch.nn.Dropout(self.dropout)
        
        # Linear layers for the attention mechanism
        self.V = torch.nn.Linear(self.num_hidden,1)
        
        # layer to get the pointer probability
        self.generator = torch.nn.Linear(2*self.num_hidden+self.embed_dims,1)
        # output layer to vocab
        self.output_layer = torch.nn.Linear(2*self.num_hidden,self.vocab_size)
        # softmax function
        self.softmax = torch.nn.Softmax(dim=1)
        # decoder LSTM
        self.lstm = torch.nn.LSTM(input_size = self.num_hidden+self.embed_dims,hidden_size = self.num_hidden,
                                  num_layers = self.layers,batch_first=True,dropout=0.5,
                                 bidirectional=False)
        # embedding matrix
        self.embedding = create_emb(weight_matrix,True)
        self.sig = torch.nn.Sigmoid()
        self.device = device
        
        
    def forward(self,x,enc_out,hidden,text,batch_size):
        # decoder
        # Decoder Input Shape: [batch_size]
        x = self.embedding(x).unsqueeze(1)
        # Decoder Embedded Shape: [batch_size,1,embed_dim]
        x = self.dropout_layer(x)
        
        # Bahdanau Attention
        dec_a = hidden[0].permute(1,0,2)
        enc_score = self.V(torch.tanh(enc_out + dec_a)) # attention score
        # Attention Score Shape: [batch_size,input_seq_len,1]
        enc_weight = self.softmax(enc_score) # attention weight
        # Attention Weight Shape: [batch_size,input_seq_len,1]
        enc_context = torch.mul(enc_weight,enc_out) # find the context vector
        # Attention Context Shape: [batch_size,input_seq_len,decoder_num_hidden]
        enc_context = enc_context.sum(1)
        # Attention Context Shape: [batch_size,decoder_num_hidden]
        enc_context.unsqueeze_(1)
        # Attention Context Shape: [batch_size,1,decoder_num_hidden]     
        
        d_in = torch.cat((x,enc_context),2)
        # Decoder Input Shape: [batch_size,1,decoder_num_hidden+embed_dims]  
        
        # run the decoder LSTM
        d_output, hidden = self.lstm(d_in,hidden)
        # Decoder Output Shape: [batch_size,1,decoder_num_hidden]  
        # Decoder Hidden Shape: [1,batch_size,decoder_num_hidden]  

        # concatenate output with the encoder context tensor
        output = torch.cat((d_output.squeeze(1),enc_context.squeeze(1)),1)
        # Decoder Output Shape: [batch_size,2*decoder_num_hidden]  
        
        output_generator = torch.cat((enc_context.squeeze(1),d_output.squeeze(1),x.squeeze(1)),1)
        # Generator Input Shape: [batch_size,2*decoder_num_hidden + embed_dims]  
        
        p_gen = self.sig(self.generator(output_generator).squeeze(1))
        
        # pointer-generator
        p_pointer = 1 - p_gen
        pointer_prob = torch.zeros([batch_size,self.vocab_size],device=self.device)
        for i in range(batch_size):
            pointer_prob[i,text[i,:]] = enc_weight[i,:,0] # pointer probability weights are the attention scores
        generator_prob = self.output_layer(output) # output layer to get vocabulary probability
        output_probability = torch.mul(p_pointer.unsqueeze(1),pointer_prob) + torch.mul(p_gen.unsqueeze(1),generator_prob)
        
        return output_probability, hidden
    
    def init_hidden(self,batch_size):
        # initialize the hidden state as the zero tensor
        weight = next(self.parameters()).data
        hidden = (weight.new(self.layers, batch_size, 2*self.num_hidden).zero_(),
                  weight.new(self.layers, batch_size, 2*self.num_hidden).zero_())
        return hidden

In [ ]:
# hyperparameters of the mode
kernel_size = [1,3,5] # kernel size for CNN
filter_size = 100 # number of kernels used in CNN
dropout = 0.5 # dropout probability
num_hidden = 256 # number of hidden units in LSTMs
enc_layers = 1 # number of layers in the LSTM encoder
batch_size = 32
vocab_size = len(word2idx) # length of vocab
dec_layers = 1 # number of layers in the LSTM decoder
embed_dims = 300 # embedding dimensions

# create a beam search node to store the running sequences for the beam search decoder
# and records the hidden state associated with the sequence, the probability and the loss
class BeamNode(object):
    def __init__(self,hidden_state,seq,prob,length,loss):
        self.hidden = hidden_state
        self.seq = seq
        self.prob = prob
        self.len = length
        self.loss = loss
        self.s = self.score()
        
    def score(self):
        return self.prob/float(self.len-1+1e-6) # calculates the score of a sequence

class Seq2Seq(nn.Module):
    def __init__(self,kernel_size,filter_size,dropout,num_hidden,enc_layers,weight_embedding,
                vocab_size,dec_layers,embed_dims,device):
        super(Seq2Seq, self).__init__()
        self.vocab_size = vocab_size
        self.device = device
        self.encoder = Encoder(kernel_size,filter_size,dropout,num_hidden,enc_layers,weight_embedding,embed_dims,device)
        self.decoder = AttentionDecoder(2*num_hidden,dropout,vocab_size,dec_layers,weight_embedding,embed_dims,device)
    
    def forward(self,x,target,e_hidden,criterion,batch_size):
        # training the decoder
        loss = 0
        prediction = target[:,0].unsqueeze(1) # records the running sequences generated by the decoder
        enc_output,enc_hidden = self.encoder(x,e_hidden) # run the encoder
        d_hidden = enc_hidden # the decoder input hidden state is the encoder output hidden state
        dec_input = target[:,0] # the decoder input starts of as the 'bos' token
        for t in range(1,target.shape[1]):
            # run the decoder
            logits, d_hidden = self.decoder(dec_input,enc_output,d_hidden,x,batch_size)
            dec_input = target[:,t] # teacher forcing turned on 
            loss += criterion(logits,target[:,t]) # calculate the loss function
            # add to get the running prediction output by the decoder
            prediction = torch.cat((prediction,torch.argmax(logits,dim=0).unsqueeze(1)),0)
        return loss, prediction
    
    def inference_greedy(self,x,target,e_hidden,criterion,batch_size):
        loss = 0
        prediction = target[:,0].unsqueeze(1) # records the running sequences generated by the decoder
        enc_output,enc_hidden = self.encoder(x,e_hidden) # run the encoder
        d_hidden = enc_hidden # the decoder input hidden state is the encoder output hidden state
        dec_input = target[:,0] # the decoder input starts of as the 'bos' token
        for t in range(1,target.shape[1]):
            # run the decoder
            logits, d_hidden = self.decoder(dec_input,enc_output,d_hidden,x,batch_size)
            # the input to the decoder at the next step is the argument with the largest probability
            dec_input = torch.argmax(logits,dim=1)
            loss += criterion(logits,target[:,t]) # calculate the loss
            # add to get the running prediction output by the decoder
            prediction = torch.cat((prediction,torch.argmax(logits,dim=0).unsqueeze(1)),0)
        return loss, prediction
    
    def inference_beam(self,x,target,e_hidden,criterion,beam_width,batch_size):
        decoded = [] 
        losses = 0
        enc_output,enc_hidden = self.encoder(x,e_hidden) # run the encoder
        for i in range(batch_size):
            # for each sentence in the batch
            prediction = target[i,0].view([1]).unsqueeze(1) # running prediction tensor
            dec_hidden = enc_hidden[0].permute(1,0,2)[i,:,:].unsqueeze(0)
            dec_input = target[i,0].view([1])
            d_hidden = (enc_hidden[0][:,i,:].unsqueeze(0),enc_hidden[1][:,i,:].unsqueeze(0))
            first_node = BeamNode(d_hidden,dec_input,0,1,0) # the first node is the 'bos' token
            nodes = queue.PriorityQueue(maxsize=beam_width) # create a priority queue to store the beam search nodes
            nodes.put((-first_node.score(),first_node)) # place the first node in the queue
            for t in range(1,target.shape[1]):
                # go through each of the words in the target
                candidatenodes = [] # stores the candidate nodes for a one step look ahead
                candidatescore = [] # stores the candidate scores for a one step look ahead
                donenodes = [] # stores the sequences that have been completed - contain 'eos' token

                # This runs through the 10 sequences in the queue and does a one step look ahead
                # to find 100 possible candidates. The top 10 candidates are then added to the queue
                # and the next word of the decoder is computed
                
                for _ in range(nodes.qsize()):
                    # for each sequence in the queue
                    sc,nodex = nodes.get() # get the node from the queue
                    seq = nodex.seq.view([-1,1]) # the sequence in the node
                    dec_input = seq[-1,:] # the last word of the sequence is the input to the decoder
                    hidden = nodex.hidden # the hidden state associated with the sequence
                    d_hidden = (hidden[0],hidden[1]) # the hidden state of the decoder
  
                    if dec_input == word2idx['end']:
                        # if the sequence is completed, then it is added to 'donenodes' list
                        donenodes.append((sc,nodex))
                    else:
                        # run the decoder
                        logits, d_hidden = self.decoder(dec_input,enc_output[i,:,:].unsqueeze(0),d_hidden,x,batch_size=1)
                        # calculate the loss
                        loss = criterion(logits,target[i,t].view([1]))
                        # pick the top 10 logits values
                        log_p, index = torch.topk(logits,beam_width)
                        for k in range(beam_width):
                            # create a beam node for each of the top 10 to get the candidate nodes
                            node = BeamNode(d_hidden, torch.cat([nodex.seq,index[0,k].unsqueeze(0)]),nodex.prob+log_p[0][k],nodex.len+1,nodex.loss+loss)
                            score = -node.score()
                            candidatenodes.append([score,node])
                # put the finished sequences first in the queue first and then fill with candidate nodes
                for score,n in donenodes:
                    nodes.put((score,n))
                for score,n in candidatenodes[nodes.qsize():beam_width]:
                    nodes.put((score,n))
            _,output_node = nodes.get()
            
            # the output of the beam search decoder is the sequence with the lowest score
            decoded.append(output_node.seq)
            losses+=output_node.loss
            del nodes
        return losses/batch_size,decoded

In [ ]:
def get_batches(x, y,batch_size=100):
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [ ]:
import time

epochs = 2
learning_rate = 0.001
# initialize model
model = Seq2Seq(kernel_size,filter_size,dropout,num_hidden,enc_layers, torch.from_numpy(weight_embedding),
                vocab_size,dec_layers,embed_dims,device).to(device)
# Adam Optimizer
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
# cross entropy loss function with the padding ignored
criterion = nn.CrossEntropyLoss(ignore_index=0)
counter = 0
loss = []
beam_width = 10 # beam search width is 10
for e in range(epochs):
    start_time = time.time() # start timer
    # shuffle the data at the beginning of epoch
    p = np.random.permutation(train_ans.shape[0])
    train_ques = train_ques[p,:]
    train_ans = train_ans[p,:]
    # initialize encoder initial hidden state
    e_hidden = model.encoder.init_hidden(batch_size)
    
    for x,y in get_batches(train_ques,train_ans,batch_size):
        model.train()
        # convert inputs to PyTorch tensor
        x = torch.from_numpy(x).to(device)
        y = torch.from_numpy(y).to(device)
        
        e_hidden = tuple([each.data for each in e_hidden])
        optimizer.zero_grad() # zero the gradients of the model
        # train the model
        l,prediction = model(x,y,e_hidden,criterion,batch_size)
        loss.append(l.item()) # add loss function
        l.backward()
        nn.utils.clip_grad_norm_(model.parameters(),2) # gradient clip with norm = 2
        optimizer.step() # step the optimizer to apply the gradient
        
        if counter%50 == 0:
            print("Epoch: {}/{} ".format(e+1, epochs),
                      "\tStep: {} ".format(counter),
                      "\tLoss: {:.4f} ".format(l.item()))
        counter += 1
    print("Time to train epoch: {0} s".format(time.time()-start_time))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:63: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch: 1/2  	Step: 0  	Loss: 150.0630 
Epoch: 1/2  	Step: 50  	Loss: 111.5572 


KeyboardInterrupt: ignored

In [ ]:
with torch.no_grad():
    beam_time = time.time() # start timer
    loss_beam = [] 
    beam_predict = [] # save beam search decoder outputs
    ans_validation = [] # save validation target summaries
    ques_validation = [] # save validation input text
    model.eval()
    # initialize the encoder hidden state
    val_hidden = model.encoder.init_hidden(batch_size)
    for x_val, y_val in get_batches(val_ques,val_ans,batch_size):
        # convert data to PyTorch tensor
        x_val = torch.from_numpy(x_val).to(device)
        y_val = torch.from_numpy(y_val).to(device)
        val_hidden = tuple([each.data for each in val_hidden])
        # run the beam search decoder
        val_loss, prediction = model.inference_beam(x,y,val_hidden,criterion,beam_width,batch_size=32)
        print(prediction)
        loss_beam.append(val_loss.item())
        beam_predict.append(prediction)
        ans_validation.append(y_val)
        ques_validation.append(x_val)
        break
    model.train()
    print("Beam Test: {0} s".format(time.time()-beam_time))
    print("Val Beam Search Loss: {:.4f}".format(np.mean(loss_beam)))


[tensor([82952,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1], device='cuda:0'), tensor([7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0'), tensor([9, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0'), tensor([3627,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1], device='cuda:0'), tensor([690,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1],
       device='cuda:0'), tensor([91313,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1], device='cuda:0'), tensor([2063,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1], device='cuda:0'), tensor([83,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
       device='cuda:0'), tensor([11982,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1], device='cuda:0'), tensor([4, 1, 1, 1, 1, 1, 1, 1

In [ ]:
beam = []
file = open("beam.txt","w")
for i in beam_predict:
    for p in i:
        array = torch.Tensor.cpu(p).numpy()
        beam_string = ''
        for q in array:
            beam_string += idx2word[q] + ' ' 
        file.write(beam_string+'\n')
        beam.append(beam_string)
file.close()

# save the validation summaries for the input text used in the beam search decoder
summary = []
file = open("validation_answer.txt","w")
for i in ans_validation:
    for p in i:
        array = torch.Tensor.cpu(p).numpy()
        summary_string = ''
        for q in array:
            if q != 0:
                summary_string += idx2word[q] + ' ' 
        file.write(summary_string+'\n')
        summary.append(summary_string)
file.close()

# Save the validation input text as a .txt file
text = []
file = open("validation_question.txt","w")
for i in ques_validation:
    for p in i:
        array = torch.Tensor.cpu(p).numpy()
        text_string = ''
        for q in array:
            if q != 0:
                text_string += idx2word[q] + ' ' 
        file.write(text_string+'\n')
        text.append(text_string)
file.close()

Deepika 
Deepika the 
Deepika the the 
Deepika the the the 
Deepika the the the the 
Deepika the the the the the 
Deepika the the the the the the 
Deepika the the the the the the the 
Deepika the the the the the the the the 
Deepika the the the the the the the the the 
Deepika the the the the the the the the the the 
Deepika the the the the the the the the the the the 
Deepika the the the the the the the the the the the the 
Deepika the the the the the the the the the the the the the 
and 
and the 
and the the 
and the the the 
and the the the the 
and the the the the the 
and the the the the the the 
and the the the the the the the 
and the the the the the the the the 
and the the the the the the the the the 
and the the the the the the the the the the 
and the the the the the the the the the the the 
and the the the the the the the the the the the the 
and the the the the the the the the the the the the the 
he 
he the 
he the the 
he the the the 
he the the the the 
he the the the t

In [ ]:
def answer_generation(ques):
  sentence = []
  for word in ques.split():
      if word in word2idx:
        sentence.extend([word2idx[word]])
      else:
        word = word.lower()
        if word in word2idx:
          sentence.extend([word2idx[word]])
        else:
          if word == 'eos':
            sentence.extend([word2idx['end']])
          else:
            sentence.extend([word2idx['unknown']])
  ques_len = 50 
  ques_feature = np.zeros((1, 50))
  ques_feature[:,0:len(sentence)] = np.array(sentence)[:50]


In [ ]:
answer_generation("What is your name ?")

(1, 50)
